In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

In [2]:
# Load the CSV file into a pandas DataFrame
df_apps = pd.read_csv('data/apps_sample.csv')
df_transactions = pd.read_csv('data/txs_sample.csv')

In [3]:
# Convert the date columns to datetime objects + other initial data cleaning
df_transactions['date'] = pd.to_datetime(df_transactions['date'], errors='coerce').dt.date
df_apps['issuanceDate'] = pd.to_datetime(df_apps['issuanceDate'], errors='coerce').dt.date
df_apps['applicationDate'] = pd.to_datetime(df_apps['applicationDate'], errors='coerce').dt.date
df_apps['repaidDate'] = pd.to_datetime(df_apps['repaidDate'], errors='coerce').dt.date
df_apps['nextPaycheck'] = df_apps['nextPaycheck'].str.replace('T.*', '', regex=True)
df_apps['nextPaycheck'] = pd.to_datetime(df_apps['nextPaycheck'], errors='coerce').dt.date
def parse_tags(x):
    if pd.isna(x):  # Handle NaN
        return []
    try:
        return ast.literal_eval(x)  # Convert string to list
    except (ValueError, SyntaxError):  # Handle invalid formats
        return []
df_transactions['tags'] = df_transactions['tags'].apply(parse_tags)
df_transactions['tags'] = df_transactions['tags'].apply(lambda x: x if isinstance(x, list) else [])
df_apps["paidByUser"] = df_apps["paidByUser"].astype(bool)

In [7]:
def extract_loan_transactions(email: str) -> list:
    # Get the user's loan applications
    user_apps = df_apps[df_apps['email'] == email].sort_values(by='applicationDate')
    # Get the user's transactions
    user_txs = df_transactions[df_transactions['email'] == email].sort_values(by='date')
    # Get the user's loan transactions
    loans = []
    for apps_index, apps_row in user_apps.iterrows():
        if apps_row['status'] == 'Repaid':
            for txs_index, txs_row in user_txs.iterrows():
                if txs_row['amount'] == apps_row['paidAmount'] and txs_row['date'] == apps_row['repaidDate']:
                    loans.append((apps_row, txs_row))
                    break
        elif apps_row['status'] == 'AutoRejected':
            loans.append((apps_row, []))
    return loans

In [80]:
# Get all the transactions for a specific email and a in a specific date range
def get_transactions_for_dates(email: str, start_date: str, end_date: str) -> pd.DataFrame:
    # Get the user's transactions
    user_txs = df_transactions[df_transactions['email'] == email]
    # Filter the transactions by date
    user_txs = user_txs[(user_txs['date'] >= start_date) & (user_txs['date'] <= end_date)]
    return user_txs

def get_initial_balance(email: str, loan_row) -> float:
    # Get the user's transactions
    user_txs = df_transactions[df_transactions['email'] == email]
    # Filter the transactions by date
    user_txs = user_txs[user_txs['date'] <= loan_row['applicationDate']]
    return user_txs.iloc[-1]['balance'] if not user_txs.empty else 0

In [109]:
def data_for_timestep(email: str, loans: list, currIdx: int):
    loan_row, tx_row = loans[currIdx]
    if loan_row['status'] == 'AutoRejected':
        return None
    start_date = loan_row['applicationDate']
    end_date = loan_row['repaidDate']
    # Get the user's transactions
    amount_requested = loan_row['amount']
    user_txs = get_transactions_for_dates(email, pd.to_datetime('2024-01-01').date(), start_date)
    user_txs_during_loan = get_transactions_for_dates(email, start_date, end_date)
    # Get the initial balance
    initial_balance = get_initial_balance(email, loan_row)
    txs = get_transactions_for_dates(email, pd.to_datetime('2024-01-01').date(), loan_row['issuanceDate'])
    recent_txs = txs[txs['date'] >= max(loan_row['issuanceDate'] - pd.Timedelta(days=30), pd.Timestamp('2024-01-01').date())]
    avg_amount_spent_last_30_days = np.mean(recent_txs['amount']) if not recent_txs.empty else 0
    std_amount_spent_last_30_days = np.std(recent_txs['amount']) if not recent_txs.empty else 0
    avg_spending = np.mean(user_txs['amount']) if len(user_txs) > 0 else 0
    avg_balance = np.mean(user_txs['balance']) if len(user_txs) > 0 else 0
    days_since_last_loan_request = (start_date -loans[currIdx-1][0]['applicationDate']).days if currIdx > 0 else -1
    was_last_loan_auto_rejected = loans[currIdx-1][0]['status'] == 'AutoRejected' if currIdx > 0 else False
    
    avg_num_txs_per_day_during_loan = len(user_txs_during_loan) / ((end_date - start_date).days + 1) if len(user_txs_during_loan) > 0 else 0
    avg_daily_balance_during_loan = np.mean(user_txs_during_loan['balance']) if len(user_txs_during_loan) > 0 else 0
    std_daily_balance_during_loan = np.std(user_txs_during_loan['balance']) if len(user_txs_during_loan) > 0 else 0
    avg_spending_amount_during_loan = np.mean(user_txs_during_loan['amount']) if len(user_txs_during_loan) > 0 else 0
    std_spending_amount_during_loan = np.std(user_txs_during_loan['amount']) if len(user_txs_during_loan) > 0 else 0
    num_times_balance_below_0 = len(user_txs_during_loan[user_txs_during_loan['balance'] < 0])
    balance_after_repayment = tx_row['balance'] if not tx_row.empty else 0
    amount_paid = tx_row['amount'] if not tx_row.empty else 0
    voluntary_repayment = loan_row['paidByUser'] if not loan_row.empty else False
    how_early_repayment = max(abs((loan_row['repaidDate'] - loan_row['nextPaycheck']).days), 0) if not loan_row.empty else 0
    return [amount_requested, initial_balance, avg_balance, avg_spending, avg_amount_spent_last_30_days, std_amount_spent_last_30_days, days_since_last_loan_request, was_last_loan_auto_rejected, avg_num_txs_per_day_during_loan, avg_daily_balance_during_loan, std_daily_balance_during_loan, avg_spending_amount_during_loan, std_spending_amount_during_loan, num_times_balance_below_0, balance_after_repayment, amount_paid, voluntary_repayment, how_early_repayment]


In [110]:
def data_for_input_timestep(email: str, loans: list, currIdx: int):
    loan_row, tx_row = loans[currIdx]
    if loan_row['status'] == 'AutoRejected':
        return None
    start_date = loan_row['applicationDate']
    # Get the user's transactions
    amount_requested = loan_row['amount']
    user_txs = get_transactions_for_dates(email, pd.to_datetime('2024-01-01').date(), start_date)
    # Get the initial balance
    initial_balance = get_initial_balance(email, loan_row)
    txs = get_transactions_for_dates(email, pd.to_datetime('2024-01-01').date(), loan_row['issuanceDate'])
    recent_txs = txs[txs['date'] >= max(loan_row['issuanceDate'] - pd.Timedelta(days=30), pd.Timestamp('2024-01-01').date())]
    avg_amount_spent_last_30_days = np.mean(recent_txs['amount']) if not recent_txs.empty else 0
    std_amount_spent_last_30_days = np.std(recent_txs['amount']) if not recent_txs.empty else 0
    avg_spending = np.mean(user_txs['amount']) if len(user_txs) > 0 else 0
    avg_balance = np.mean(user_txs['balance']) if len(user_txs) > 0 else 0
    days_since_last_loan_request = (start_date -loans[currIdx-1][0]['applicationDate']).days if currIdx > 0 else -1
    was_last_loan_auto_rejected = loans[currIdx-1][0]['status'] == 'AutoRejected' if currIdx > 0 else False
    output = [amount_requested, initial_balance, avg_balance, avg_spending, avg_amount_spent_last_30_days, std_amount_spent_last_30_days, days_since_last_loan_request, was_last_loan_auto_rejected]
    output.extend([0]*10)
    return output

In [111]:
def format_one_data_point(email: str):
    loans = extract_loan_transactions(email)
    timesteps = []
    timesteps_inputs = []
    data = []
    labels =  []
    for i in range(len(loans)):
        data_point = data_for_timestep(email, loans, i)
        if data_point is not None:
            timesteps.append(data_point)
        data_point_input = data_for_input_timestep(email, loans, i)
        if data_point_input is not None:
            timesteps_inputs.append(data_point_input)
            labels.append(loans[i][1]['balance'])
    for i in range(0, len(timesteps)):
        data.append(timesteps[:i] + [timesteps_inputs[i]])
    return data, labels

In [112]:
res = format_one_data_point('61be9c6909d91d74fe46a2c9f6d830d69f7015e8fa2ac4a4129cbb3edd18ae21')
data = res[0]
labels = res[1]

TypeError: unsupported operand type(s) for +: 'datetime.timedelta' and 'int'

In [106]:
data[39]

[[35.0,
  np.float64(18.04),
  np.float64(121.77144144144147),
  np.float64(0.26315315315315435),
  np.float64(-0.5305882352941149),
  np.float64(159.1083693022052),
  -1,
  False,
  2.388888888888889,
  np.float64(100.8618604651163),
  np.float64(185.59031902949275),
  np.float64(0.24244186046511887),
  np.float64(132.216764267394),
  3,
  541.1,
  45.24,
  False,
  0],
 [40.0,
  np.float64(18.04),
  np.float64(111.98025252525252),
  np.float64(0.2528282828282853),
  np.float64(0.10698412698412928),
  np.float64(111.62312304733736),
  37,
  False,
  1.0357142857142858,
  np.float64(205.89448275862065),
  np.float64(213.72161187711725),
  np.float64(-2.620344827586205),
  np.float64(172.08903181869226),
  2,
  367.62,
  47.99,
  False,
  0],
 [45.0,
  np.float64(18.04),
  np.float64(125.08288888888889),
  np.float64(0.04031111111111285),
  np.float64(-0.8658536585365826),
  np.float64(196.96329614714847),
  28,
  False,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [107]:
def get_data():
    data = []
    labels = []
    count = 0
    unique_emails = df_transactions['email'].unique()
    unique_emails_list = unique_emails.tolist()
    for email in unique_emails_list:
        res = format_one_data_point(email)
        data.extend(res[0])
        labels.extend(res[1])
        count += 1
        if count % 10 == 0:
            print(count)
        if count == 50:
            break
    return data, labels

In [108]:
data, labels = get_data()

10
20
30
40
50
60
70
80


ZeroDivisionError: division by zero

In [ ]:
data